In [1]:
import __init__
import models.cross_validator as cross_validator
import models.datahandler as datahandler
from models.datamodel import DataModel

trial = datahandler.load_train('../data/dataset/tsd_trial.csv', verbose=True)
train = datahandler.load_train('../data/dataset/tsd_train.csv', verbose=True)

datamodel = DataModel(model='cmm', split_toxics=True)

/home/hb/.local/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


,spans,text
0,"[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",Because he's a moron and a bigot. It's not any...
1,"[29, 30, 31, 32, 33, 34]",How about we stop protecting idiots and let na...
2,"[166, 167, 168, 169, 170, 171]","If people were smart, they would Boycott th..."


,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."


In [14]:
X_train, y_train, train_taboo_words, train_texts = cross_validator.load_data(train, 
                                                                            [i for i in range(train.shape[0])], 
                                                                            datamodel, logger=True)

X_test, y_test, test_taboo_words, test_texts = cross_validator.load_data(trial, 
                                                                         [i for i in range(trial.shape[0])], 
                                                                         datamodel, logger=True)

100%|██████████| 690/690 [00:48<00:00, 14.19it/s]


In [15]:
import pandas as pd
from tqdm import tqdm

def create_json(X, Y, texts, taboo_words):
    jsn = {}
    for index in range(len(X)):
        jsn[index]={
            'text':texts[index],
            'truth-spans':Y[index],
            'x-test':X[index],
            'taboo-words':taboo_words[index]
        }
    return jsn

def create_df(X, Y, texts, taboo_words):
    column = ['index', 'text', 'grand-truth', 'taboo-words', 'token', 
              'token-tag', 'token-iob', 'token-span', 'class']
    data= []
    for index in tqdm(range(len(X))):
        tokens, spans, text, taboo_word = X[index], Y[index], texts[index], taboo_words[index]
        for item in tokens:
            token, tag, iob, label, span = item
            data.append([index, text, spans, taboo_word, token, tag,iob, span, label])
    df = pd.DataFrame(data = data, columns= column)
    return df

In [18]:
trial_df = create_df(X_test, y_test, test_texts, test_taboo_words)

train_df = create_df(X_train, y_train, train_texts, train_taboo_words)

100%|██████████| 7939/7939 [00:00<00:00, 25829.77it/s]


In [20]:
trial_df.to_csv("../data/datast-v2/trial-token-df.csv", index=False)

train_df.to_csv("../data/datast-v2/train-token-df.csv", index=False)